Core components

- chains
- components
- agents
- indexes

In [2]:
!pip install --quiet langchain openai

In [8]:
# setting the open ai key

from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Basic setup

In [10]:
# predicting a sample one

from langchain.llms import OpenAI

def generate_pet_name():
  llm = OpenAI(temperature=0.7)

  name = llm("generate 5 names for my pet")
  return name

print(generate_pet_name())



1. Marshmallow
2. Whiskers
3. Luna
4. Biscuit
5. Gizmo


### With Chains

In [16]:
# prompt template with chain

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def generate_pet_name(animal_type):
  llm = OpenAI(temperature=0.7)

  prompt_template = PromptTemplate(
      input_variables=["animal_type"],
      template = "generate 5 names for my pet {animal_type}",
  )

  llm_chain = LLMChain(
      llm=llm,
      prompt=prompt_template,
  )
  response = llm_chain({
      "animal_type": animal_type,
  })
  return response["text"]

print(generate_pet_name("dog"))



1. Max
2. Bailey
3. Luna
4. Duke
5. Coco


### Agents

In [21]:
! pip install wikipedia --quiet

In [25]:
from langchain.agents import load_tools, initialize_agent, AgentType

def langchain_agent():
  llm = OpenAI(temperature=0.5)

  tools = load_tools(["wikipedia", "llm-math"], llm=llm)

  agent = initialize_agent(
      tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
  )

  result = agent.run(
      "What is the normal age of a dog in wikipedia? add 5 to the age"
  )
  print(result)

langchain_agent()



> Entering new AgentExecutor chain...
 I should try to find the normal age of a dog and then add 5 to it.
Action: Wikipedia
Action Input: Normal age of a dog
Observation: Page: Dog whistle
Summary: A dog whistle (also known as silent whistle or Galton's whistle) is a type of whistle that emits sound in the ultrasonic range, which humans cannot hear but some other animals can, including dogs and domestic cats, and is used in their training. It was invented in 1876 by Francis Galton and is mentioned in his book Inquiries into Human Faculty and Its Development, in which he describes experiments to test the range of frequencies that could be heard by various animals, such as a house cat.
The upper limit of the human hearing range is about 20 kilohertz (kHz) for children, declining to 15–17 kHz for middle-age adults.  The top end of a dog's hearing range is about 45 kHz, while a cat's is 64 kHz.  It is thought that the wild ancestors of cats and dogs evolved this higher hearing range in o

### Indexing


In [46]:
!pip install youtube-transcript-api tiktoken faiss-cpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.5 MB/s eta 0:00:00


In [47]:
# chat with a youtube content

from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS # a library for similarity search

In [48]:
# setting up a temporary vector db

embeddings = OpenAIEmbeddings()

def create_db_from_yt(url:str) -> FAISS:
  loader = YoutubeLoader.from_youtube_url(url)
  transcript = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 1000,
      chunk_overlap = 100,
  )
  docs = text_splitter.split_documents(transcript)

  db = FAISS.from_documents(docs, embeddings)
  return db

In [55]:
# setting up the main method

template = """
    You are a helpful assistant that that can answer questions about youtube videos
    based on the video's transcript.

    Answer the following question: {question}
    By searching the following video transcript: {docs}

    Only use the factual information from the transcript to answer the question.

    If you feel like you don't have enough information to answer the question, say "I don't know".

    Your answers should be verbose and detailed."""

def get_response(db, query, k=3):
  docs = db.similarity_search(query, k=k)
  docs_page_content = " ".join([d.page_content for d in docs])

  llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")
  prompt = PromptTemplate(
      input_variables = ["question", "docs"],
      template=template,
  )
  chain = LLMChain(
      llm=llm,
      prompt=prompt,
  )
  response = chain.run(
      question = query,
      docs = docs_page_content,
  )
  response = response.replace("\n", "")
  return response, docs

In [56]:
# running the thing

db = create_db_from_yt("https://www.youtube.com/watch?v=M-ZH3psUbfU")
query = "what is the best advice for men ?"

res, docs = get_response(
    db=db,
    query=query,
)
print(res)

The best advice for men is to try to be useful and contribute positively to society. This was mentioned multiple times in the video transcript, with the speaker encouraging people to live a useful life and have a positive net contribution to society. This can be achieved by finding something that one is passionate about and using their talents to make a difference in that area. The speaker also mentions that it is important to have a good general knowledge and be well-informed about various subject areas, as this can help one discover their interests and talents. Additionally, the speaker suggests reading broadly and learning from different sources, such as books and conversations with people from different backgrounds and professions. This can help one gain a better understanding of the world and find something that they are truly interested in and can excel at. The speaker also advises against pursuing leadership positions for the sake of being a leader, as true leaders are often tho

In [57]:
# printing in a good way
from pprint import pprint

pprint(res)

('The best advice for men is to try to be useful and contribute positively to '
 'society. This was mentioned multiple times in the video transcript, with the '
 'speaker encouraging people to live a useful life and have a positive net '
 'contribution to society. This can be achieved by finding something that one '
 'is passionate about and using their talents to make a difference in that '
 'area. The speaker also mentions that it is important to have a good general '
 'knowledge and be well-informed about various subject areas, as this can help '
 'one discover their interests and talents. Additionally, the speaker suggests '
 'reading broadly and learning from different sources, such as books and '
 'conversations with people from different backgrounds and professions. This '
 'can help one gain a better understanding of the world and find something '
 'that they are truly interested in and can excel at. The speaker also advises '
 'against pursuing leadership positions for the sak

That's it. thanks a loottt !